<a href="https://colab.research.google.com/github/karchx/ml_101/blob/main/nlp_polizas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!java --version

In [1]:
!pip install tabula-py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 27.2 MB/s eta 0:00:00


In [2]:
import tabula
import numpy
import os

tabula.environment_info()
numpy.__version__

os.path.abspath("")


Python version:
    3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
Java version:
    openjdk version "11.0.25" 2024-10-15
OpenJDK Runtime Environment (build 11.0.25+9-post-Ubuntu-1ubuntu122.04)
OpenJDK 64-Bit Server VM (build 11.0.25+9-post-Ubuntu-1ubuntu122.04, mixed mode, sharing)
tabula-py version: 2.10.0
platform: Linux-6.1.85+-x86_64-with-glibc2.35
uname:
    uname_result(system='Linux', node='d339df26ba60', release='6.1.85+', version='#1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024', machine='x86_64')
linux_distribution: ('Ubuntu', '22.04', 'jammy')
mac_ver: ('', ('', '', ''), '')


'/content'

In [79]:
!pip freeze | grep openpyxl

openpyxl==3.1.5


In [94]:
import tabula
import pandas as pd

def extract_data(path: str):
    dfs = tabula.read_pdf(path, stream=True, pages=0)
    df = dfs[0]
    df.columns = [f"column_{i}" for i in range(len(df.columns))]

    columns = []
    values = []

    for col in df.columns:
        for cell in df[col].dropna():
            # GET Poliza
            if "Póliza" in cell:
              parts = cell.split(" ")
              if len(parts) == 2:
                key, value = parts
                columns.append(key.strip())
                clean_value = value.replace("No.", "").strip()
                values.append(clean_value)

            if "Q" in cell:
                parts = cell.split("Q ")
                if len(parts) == 2:
                    key, value = parts
                    columns.append(key.strip())
                    values.append(value.strip())

    # Get Suma asegurada
    df_suma = df[df['column_0'] == "Suma Asegurada:"]
    key = df_suma['column_0'].values.astype(str)[0]
    key = key.replace(":", "").strip()
    value = df_suma['column_1'].values.astype(str)[0]
    value = value.replace("Q.", "").strip()
    columns.append(key.strip())
    values.append(value.strip())

    df_final = pd.DataFrame(columns=columns, data=[values])

    print(df_final.to_json(orient="records", force_ascii=False))

In [95]:
pdf_path = "./test.pdf"
extract_data(pdf_path)

[{"Póliza":"INFA-1038","Prima":"158,400.00","Gtos. Emisión":"7,920.00","Gtos. Fracc.":"0.00","Otros Gastos":"0.00","Dto. 1422":"3,160.00","IVA":"19,958.40","Total":"189,438.40","Suma Asegurada":"132,000,000.00"}]


In [5]:
import re

In [6]:
def show_image(item, title=""):
    """Display a pixmap.

    Just to display Pixmap image of "item" - ignore the man behind the curtain.

    Args:
        item: any PyMuPDF object having a "get_pixmap" method.
        title: a string to be used as image title

    Generates an RGB Pixmap from item using a constant DPI and using matplotlib
    to show it inline of the notebook.
    """
    DPI = 150  # use this resolution
    import numpy as np
    import matplotlib.pyplot as plt

    # %matplotlib inline
    pix = item.get_pixmap(dpi=DPI)
    img = np.ndarray([pix.h, pix.w, 3], dtype=np.uint8, buffer=pix.samples_mv)
    plt.figure(dpi=DPI)  # set the figure's DPI
    plt.title(title)  # set title of image
    _ = plt.imshow(img, extent=(0, pix.w * 72 / DPI, pix.h * 72 / DPI, 0))

In [8]:
import fitz  # import PyMuPDF
if not hasattr(fitz.Page, "find_tables"):
    raise RuntimeError("This PyMuPDF version does not support the table feature")

In [12]:
def extract_text_from_pdf(file_path: str):
    doc = fitz.open(file_path)
    text = ""
    for page in doc:
        text += page.get_text() + "\n"
    return text


In [13]:
# TESTING

doc = fitz.open("./test_extract.pdf")
page = doc[26]

text = page.get_text("text")
pattern = r"([\w\s\.]+)Q\s+([\d,]+\.\d{2})"

matches = re.findall(pattern, text)

table_data = {item[0].strip(): float(item[1].replace(",", "")) for item in matches}
for key, value in table_data.items():
    print(f"{key}: Q {value}")

# tabs = page.find_tables()  # detect the tables
# for i,tab in enumerate(tabs):  # iterate over all tables
#     for cell in tab.header.cells:
#         print(cell)
#         page.draw_rect(cell,color=fitz.pdfcolor["red"],width=0.3)
#     page.draw_rect(tab.bbox,color=fitz.pdfcolor["green"])
#     print(f"Table {i} column names: {tab.header.names}, external: {tab.header.external}")
#
# show_image(page, f"Table & Header BBoxes")

1038
Requerimiento Serie E      No. 979476310
Total a Pagar: Q 189438.4


In [43]:
tab = tabs[0]
df = tab.to_pandas()
df

,No.,Documento Pago,Monto,Col3
0,,Efectivo,,
1,,Chqs. Propios,,
2,,Chqs. Otros Bancos,,
3,,Total,"189,438",.40


In [ ]:
data = {}
text = extract_text_from_pdf("./test_extract.pdf")
lines = text.split('\n')

for i, line in enumerate(lines):
  if "PRIMA" in line:
    data["Prima"] = lines[i + 1].split()[-1].replace(",", "")
#print(text)
print(lines)